### Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip3 install --upgrade gensim --quiet
!pip3 install fse --quiet
!pip3 install swifter --quiet
!pip3 install pickle5 --quiet

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import itertools
# import swifter
from scipy.spatial.distance import cosine
from collections import Counter
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from gensim.utils import simple_preprocess
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
import gensim.downloader
tqdm.pandas()
import nltk
nltk.download('punkt')
from IPython.display import HTML as html_print
from gensim.models.callbacks import CallbackAny2Vec
from sklearn import metrics
import swifter
import pickle5 as pickle

     |████████████████████████████████| 24.1 MB 5.9 MB/s 
     |████████████████████████████████| 90 kB 2.7 MB/s 
     |████████████████████████████████| 56.8 MB 1.1 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 596 kB 67.4 MB/s 
     |████████████████████████████████| 821 kB 4.0 MB/s 
     |████████████████████████████████| 281 kB 63.6 MB/s 
     |████████████████████████████████| 141 kB 79.6 MB/s 
     |████████████████████████████████| 256 kB 3.2 MB/s 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## **CONVERTING JSON TO CSV**

In [ ]:
def new_json_to_dataframe(file):

    f = open ( file , "r") 
    data = json.loads(f.read())               #loading the json file.
    iid = []                                  
    tit = []                                  #Creating empty lists to store values.
    con = []
    Que = []
    Ans_st = []
    Txt = []
    
    for i in range(len(data['data'])):       #Root tag of the json file contains 'title' tag & 'paragraphs' list.
        
        title = data['data'][i]['title']
        for p in range(len(data['data'][i]['paragraphs'])):  # 'paragraphs' list contains 'context' tag & 'qas' list.
            
            context = data['data'][i]['paragraphs'][p]['context']
            for q in range(len(data['data'][i]['paragraphs'][p]['qas'])):  # 'qas' list contains 'question', 'Id' tag & 'answers' list.
                
                question = data['data'][i]['paragraphs'][p]['qas'][q]['question']
                Id = data['data'][i]['paragraphs'][p]['qas'][q]['id']
                for a in range(len(data['data'][i]['paragraphs'][p]['qas'][q]['answers'])): # 'answers' list contains 'ans_start', 'text' tags. 
                    
                    ans_start = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['answer_start']
                    text = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['text']
                    
                    tit.append(title)
                    con.append(context)
                    Que.append(question)                    # Appending values to lists
                    iid.append(Id)
                    Ans_st.append(ans_start)
                    Txt.append(text)

    print('Done')      # for indication perpose.
    new_df = pd.DataFrame(columns=['Id','title','context','question','ans_start','text']) # Creating empty DataFrame.
    new_df.Id = iid
    new_df.title = tit           #intializing list values to the DataFrame.
    new_df.context = con
    new_df.question = Que
    new_df.ans_start = Ans_st
    new_df.text = Txt
    print('Done')      # for indication perpose.
    final_df = new_df.drop_duplicates(keep='first')  # Dropping duplicate rows from the create Dataframe.
    return final_df

In [ ]:
dev_data_from_json = new_json_to_dataframe('/content/drive/MyDrive/NLP Project CS6120/Code/Data/dev-v1.1.json')

train_data_from_json = new_json_to_dataframe('/content/drive/MyDrive/NLP Project CS6120/Code/Data/train-v1.1.json')

print(train_data_from_json.shape)

print(dev_data_from_json.shape)


Done
Done
Done
Done
(87599, 6)
(18216, 6)


## **BASELINE MODELLING**

### Load Dataset

In [ ]:
#path_dir =  os.path.dirname(os.getcwd())
#train_df = pd.read_csv(os.path.join(path_dir,r'data/interim/train_data.csv'))
#train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/NLP/Data/train_data.csv')
#val_df = pd.read_csv(os.path.join(path_dir,r'data/interim/val_data.csv'))
#val_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/NLP/Data/val_data.csv')
train_data_from_json = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/NLP/Data/train_data_from_json_1.csv')
dev_data_from_json = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/NLP/Data/dev_data_from_json_1.csv')
#dev_data.drop('Unnamed: 0',axis=1,inplace = True)
train_data_from_json.tail(5)

In [ ]:
def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

for i in range(1006,1009):
    print(f'\033[96m CONTEXT : {train_data_from_json["context"].tolist()[i]}')
    print(f'\033[91m QUESTION : {train_data_from_json["question"].tolist()[i]}')
    print(f'\033[92m ANSWER : {train_data_from_json["text"].tolist()[i]}')
    print('\033[90m ' + '-'*170)

 CONTEXT : Beyoncé participated in George Clooney and Wyclef Jean's Hope for Haiti Now: A Global Benefit for Earthquake Relief telethon and was named the official face of the limited edition CFDA "Fashion For Haiti" T-shirt, made by Theory which raised a total of $1 million. On March 5, 2010, Beyoncé and her mother Tina opened the Beyoncé Cosmetology Center at the Brooklyn Phoenix House, offering a seven-month cosmetology training course for men and women. In April 2011, Beyoncé joined forces with US First Lady Michelle Obama and the National Association of Broadcasters Education Foundation, to help boost the latter's campaign against child obesity by reworking her single "Get Me Bodied". Following the death of Osama bin Laden, Beyoncé released her cover of the Lee Greenwood song "God Bless the USA", as a charity single to help raise funds for the New York Police and Fire Widows' and Children's Benefit Fund.
 QUESTION : Beyonce opened a cosmetology center in what location?
 ANSWER : Br

### Get whole answer sentences

In [ ]:
def get_answer_context(df):
    length_context = 0
    answer = ""

    for sentence in sent_tokenize(df.context):
        length_context += len(sentence) + 1
        if df.ans_start <= length_context:
            if len(sentence) >= len(str(df.text)):
                if answer == "":
                    return sentence
                else:
                    return answer + " " + sentence
            else:
                answer += sentence

In [ ]:
train_data_from_json['answer_sentences'] = train_data_from_json.progress_apply(lambda row: get_answer_context(row),axis = 1)
dev_data_from_json['answer_sentences'] = dev_data_from_json.progress_apply(lambda row: get_answer_context(row),axis = 1)

100%|██████████| 18216/18216 [00:10<00:00, 1741.78it/s]


In [ ]:
train_data_from_json.head()

,Id,title,context,question,ans_start,text,answer_sentences
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"It is a replica of the grotto at Lourdes, Fran..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,Immediately in front of the Main Building and ...
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,Next to the Main Building is the Basilica of t...
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,"Immediately behind the basilica is the Grotto,..."
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,Atop the Main Building's gold dome is a golden...


### Preprocess context

In [ ]:
context_df = pd.DataFrame(train_data_from_json['context'].unique().tolist(),columns=['context'])
# Use the gensim.utils function simple_preprocess
context_df['processed'] = context_df['context'].progress_apply(lambda x: simple_preprocess(x))

question_df = pd.DataFrame(train_data_from_json['question'].unique().tolist(),columns=['question'])
# Use the gensim.utils function simple_preprocess
question_df['processed'] = question_df['question'].progress_apply(lambda x: simple_preprocess(x))


100%|██████████| 87355/87355 [00:01<00:00, 70497.61it/s]


In [ ]:
UNK = '<UNK>'

# init callback class
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
def avg_sentence_vector(words, model, num_features):
    if isinstance(model,gensim.models.word2vec.Word2Vec):
        word_vec_model = model.wv
    else:
        word_vec_model = model
    index2word_set = word_vec_model.index_to_key 
    #function to average all words vectors in a given paragraph
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in words:
        if word in index2word_set:
            nwords = nwords+1
            featureVec = np.add(featureVec, word_vec_model[word])

    if nwords>0:
        featureVec = np.divide(featureVec, nwords)
#     print(featureVec)
    return featureVec

In [ ]:
def get_cosine_similarity(context,question,model,vector_size=300):
    if isinstance(model,gensim.models.word2vec.Word2Vec):
        vocab = model.wv.key_to_index
    else:
        vocab = model.key_to_index
#     print(context,question)
    context_sents = sent_tokenize(context)
#     print(context_sents)
    processed_context = [simple_preprocess(sent) for sent in context_sents]
    processed_context = [[word if word in vocab else UNK for word in processed_context_sent]\
                         for processed_context_sent in processed_context]
#     print(processed_context)
    processed_question = simple_preprocess(question)
    processed_question = [word if word in vocab else UNK for word in processed_question]
    
    context_vectors = [np.array(avg_sentence_vector(processed_context_sent,model,vector_size)).reshape(1,-1) for processed_context_sent in processed_context]
    question_vector  = np.array(avg_sentence_vector(processed_question,model,vector_size)).reshape(1,-1)
#     print(len(context_vectors[0]))
#     print(cosine_similarity(np.array(context_vectors[0]).reshape(1,-1),np.array(question_vector).reshape(1,-1)))
    
    cosine_sim_list = [cosine_similarity(context_sent_vector,question_vector) for context_sent_vector in context_vectors]
    
#     print(f"Cosine scores: {cosine_sim_list}")
    max_cosine_sim = max(cosine_sim_list)
    predicted_answer = context_sents[np.argmax(cosine_sim_list)]
    return max_cosine_sim, predicted_answer

In [ ]:
temp_df = train_data_from_json.tail(100)

### Download word2vec model google

In [ ]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
google_model = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
sample_context = train_data_from_json['context'].tolist()[10]
sample_question = train_data_from_json['question'].tolist()[10]
print(f"\033[96m CONTEXT:{sample_context}")
print(f"\033[91m QUESTION: {sample_question}")
get_cosine_similarity(sample_context,sample_question,google_model)

 CONTEXT:The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome). Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building. Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians. Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto. The university through the Moreau Seminary has ties to theologian Frederick Buechner. While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching.
 QUESTION: Where is the headquarters of the Congregation of the Holy Cross?


(array([[0.85751235]], dtype=float32),
 'The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome).')

In [ ]:
google_model['world'].shape

(300,)

In [ ]:
google_model.most_similar(positive=['university'], topn = 3)

[('universities', 0.7003918886184692),
 ('faculty', 0.6780907511711121),
 ('unversity', 0.6758289933204651)]

### Evaluvate results

In [ ]:
def avg_sentence_vector(words, model, num_features):
    if isinstance(model,gensim.models.word2vec.Word2Vec):
        word_vec_model = model.wv
    else:
        word_vec_model = model
    index2word_set = word_vec_model.index_to_key 
    #function to average all words vectors in a given paragraph
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in words:
        if word in index2word_set:
            nwords = nwords+1
            featureVec = np.add(featureVec, word_vec_model[word])

    if nwords>0:
        featureVec = np.divide(featureVec, nwords)
#     print(featureVec)
    return featureVec
def get_context_vector(context,model,vector_size=300):
    if isinstance(model,gensim.models.word2vec.Word2Vec):
        vocab = model.wv.key_to_index
    else:
        vocab = model.key_to_index
    context_sents = sent_tokenize(context)
    processed_context = [simple_preprocess(sent) for sent in context_sents]
    processed_context = [[word if word in vocab else UNK for word in processed_context_sent]\
                         for processed_context_sent in processed_context]
    context_vectors = [np.array(avg_sentence_vector(processed_context_sent,model,vector_size)).reshape(1,-1) for processed_context_sent in processed_context]
    
    return context_vectors
    
def get_cosine_similarity(context,context_vectors,question,model,vector_size=300):
    context_sents = sent_tokenize(context)
    
    if isinstance(model,gensim.models.word2vec.Word2Vec):
        vocab = model.wv.key_to_index
    else:
        vocab = model.key_to_index
        
    processed_question = simple_preprocess(question)
    processed_question = [word if word in vocab else UNK for word in processed_question]
    
    question_vector  = np.array(avg_sentence_vector(processed_question,model,vector_size)).reshape(1,-1)
    
    cosine_sim_list = [cosine_similarity(context_sent_vector,question_vector) for context_sent_vector in context_vectors]
    
    max_cosine_sim = max(cosine_sim_list)
    predicted_answer = context_sents[np.argmax(cosine_sim_list)]
    return max_cosine_sim, predicted_answer

In [ ]:
temp_df['context_vec'] = temp_df['context']\
.swifter.progress_bar(enable=True, desc=None).apply(lambda x: get_context_vector(x,google_model))

temp_df[['cosine_sim','predicted_answer']] = temp_df[['context','context_vec','question']]\
.swifter.progress_bar(enable=True, desc=None).apply(lambda x: get_cosine_similarity(x[0],x[1],x[2],google_model,300),axis=1,result_type="expand")
temp_df.head(2)

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

,Id,title,context,question,ans_start,text,answer_sentences,context_vec,cosine_sim,predicted_answer
87499,5735c081e853931400426b42,Kathmandu,The National Museum is located in the western ...,When was the National Museum founded?,382,1928,The museum was established in 1928 as a collec...,"[[[0.041446462, 0.035127528, 0.10760498, 0.078...",[[0.676507]],The National Museum is located in the western ...
87500,5735c081e853931400426b43,Kathmandu,The National Museum is located in the western ...,What does Chhauni Silkhana mean?,505,the stone house of arms and ammunition,The museum was established in 1928 as a collec...,"[[[0.041446462, 0.035127528, 0.10760498, 0.078...",[[0.3694585]],It is the most important museum in the country...


In [ ]:
temp_df['correct_prediction'] = temp_df['answer_sentences'] == temp_df['predicted_answer']
print(temp_df['correct_prediction'].value_counts())
print(f"accuracy: {temp_df[temp_df['correct_prediction']].shape[0]/temp_df.shape[0]}")

True     66
False    34
Name: correct_prediction, dtype: int64
accuracy: 0.66


In [ ]:
# temp_df[['consine_sim','predicted_answer']] = temp_df[['context','question']]\
# .swifter.progress_bar(enable=True, desc=None)\
# .apply(lambda x: get_cosine_similarity(x[0],x[1],google_model,300),axis=1,result_type="expand")
# temp_df.head(2)

#### On val set 

In [ ]:
dev_data_from_json['context_vec'] = dev_data_from_json['context']\
.swifter.progress_bar(enable=True, desc=None).apply(lambda x: get_context_vector(x,google_model))

Pandas Apply:   0%|          | 0/18216 [00:00<?, ?it/s]

In [ ]:
dev_data_from_json[['cosine_sim','predicted_answer']] = dev_data_from_json[['context','context_vec','question']]\
.swifter.progress_bar(enable=True, desc=None)\
.apply(lambda x: get_cosine_similarity(x[0],x[1],x[2],google_model,300),axis=1,result_type="expand")
dev_data_from_json.head(2)

Pandas Apply:   0%|          | 0/18216 [00:00<?, ?it/s]

,Id,title,context,question,ans_start,text,answer_sentences,context_vec,cosine_sim,predicted_answer
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,177,Denver Broncos,The American Football Conference (AFC) champio...,"[[[-0.0418396, 0.06502618, 0.11174859, 0.08137...",[[0.8140871]],Super Bowl 50 was an American football game to...
3,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,249,Carolina Panthers,The American Football Conference (AFC) champio...,"[[[-0.0418396, 0.06502618, 0.11174859, 0.08137...",[[0.8108915]],Super Bowl 50 was an American football game to...


In [ ]:
dev_data_from_json.to_pickle('/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/NLP/Data/contextual_dev_v1.pkl')

In [ ]:

with open('/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/NLP/Data/contextual_dev_v1.pkl', "rb") as fh:
  dev_data = pickle.load(fh)

In [ ]:
dev_data['correct_prediction'] = dev_data['answer_sentences'] == dev_data['predicted_answer']
dev_data['correct_prediction'].value_counts()

True     12685
False     5531
Name: correct_prediction, dtype: int64

In [ ]:
print(f"accuracy: {dev_data[dev_data['correct_prediction']].shape[0]/dev_data.shape[0]}")

accuracy: 0.6963658322353975


Keep track of references:

1. https://www.kaggle.com/code/lidaghr/word2vec-bert-qa-diagnostics-group6-cord-19
2. https://www.kaggle.com/code/sanjay11100/squad-stanford-q-a-json-to-pandas-dataframe
3. https://www.kaggle.com/code/jagannathpatta/reading-json-data-getting-dataframe
4. https://stackoverflow.com/questions/52645459/how-to-evaluate-word2vec-model
